HistGradientBoostingClassifier<br>
Author:  Casey Kellett Whitwham



---
Imports

In [1]:
import importlib
import dataprep as dp
importlib.reload(dp)


from sklearn.metrics import accuracy_score,confusion_matrix

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import HistGradientBoostingClassifier


import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

---

Load Processed Data

In [2]:
Xtrain,ytrain,Xtest,ytest,Xdev,ydev = dp.loadprocesseddata()

(24384, 12)


c:\Users\casey\AppData\Local\Programs\Python\Python39\lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\casey\AppData\Local\Programs\Python\Python39\lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


Train X,y shape:  (14609, 84) (14609,)
Test X,y shape:  (4876, 84) (4876,)
Dev X,y shape:  (4878, 84) (4878,)


---

Define Functions

In [3]:
def gridsearch(X, y,param_grid,model):
    grid = GridSearchCV(model, param_grid,
                       scoring = "accuracy",
                       cv = 5, verbose = 1,
                       return_train_score=True,n_jobs=-1)
    grid.fit(X,y)
    print("Grid search DONE!")
    return grid



def rungrid(X_dev, y_dev,y_train, paramgrid, model):
    grid = gridsearch(X_dev, y_dev, paramgrid, model)
    df_grid_result = pd.DataFrame(grid.cv_results_)
    columns_to_show = ['params', 'rank_test_score', 'mean_train_score', 'mean_test_score']
    df_grid_result = df_grid_result[columns_to_show]
    df_grid_result = df_grid_result.sort_values(by=['rank_test_score'], ascending=True)
    mean_ytrain_prepared = np.mean(y_train)
    mean_ytest_prepared = np.mean(y_dev)
    df_grid_result.rename(columns={'mean_test_score': 'mean_dev_score'}, inplace=True)
    df_grid_result['train_error_rate'] = - 100 * df_grid_result['mean_train_score'] / mean_ytrain_prepared
    df_grid_result['dev_error_rate'] = - 100 * df_grid_result['mean_dev_score'] / mean_ytest_prepared
    top = df_grid_result.iloc[0]["params"]

    return top

---

Use top decision tree from project 1

In [4]:
top_dt ={'criterion': 'entropy','max_depth': 10,'max_features': None, 'min_samples_leaf': 4,'min_samples_split': 10, 'class_weight': None}
dt_fin =  DecisionTreeClassifier(**top_dt)

---

Define Param grid

In [5]:
#=== Hist Param_Grid ===#

paramgrid_hist = {'max_iter': [100, 200, 300],
                  'l2_regularization': [0.0, 0.1, 0.5],
                  'learning_rate':[0.01, 0.1, 0.5]}

---

Ensuring Y sets are being passed as 1d arrays

In [6]:
ytrain = np.ravel(ytrain)
ydev = np.ravel(ydev)

---

Run  GridSearch

In [7]:
histgradclass = HistGradientBoostingClassifier()
top_hist = rungrid(Xdev, ydev,ytrain, paramgrid_hist, histgradclass)
print("Top Params:",top_hist)
hist_fin = HistGradientBoostingClassifier(**top_hist)
hist_fin.fit(Xtrain,ytrain)


Fitting 5 folds for each of 27 candidates, totalling 135 fits
Grid search DONE!
Top Params: {'l2_regularization': 0.0, 'learning_rate': 0.1, 'max_iter': 300}


HistGradientBoostingClassifier(max_iter=300)

---

Evaluate Model

In [8]:
y_pred = hist_fin.predict(Xtest)
accuracy = accuracy_score(ytest, y_pred)
print("Test Accuracy:", accuracy)



Test Accuracy: 0.9302707136997539
